### Lab2 (a) Model preperation

In [1]:
from resnet20 import ResNetCIFAR
from train_util import train, finetune, test
import torch
import numpy as np

import time

import torchvision.transforms as transforms
import torchvision
import torch.nn as nn
import torch.optim as optim

from FP_layers import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
net = ResNetCIFAR(num_layers=20, Nbits=None)
net = net.to(device)

In [44]:
# Load the best weight paramters
net.load_state_dict(torch.load("pretrained_model.pt"))
test(net)

Files already downloaded and verified
Test Loss=0.3231, Test accuracy=0.9151


### Lab2 (b) Prune by percentage

In [3]:
def prune_by_percentage(layer, q=80.0):
    """
    Pruning the weight paramters by threshold.
    :param q: pruning percentile. 'q' percent of the least 
    significant weight parameters will be pruned.
    """    
    # Convert the weight of "layer" to numpy array
    np_weight = np.array(layer.weight.data.cpu())
    
    # Compute the q-th percentile of the abs of the converted array
    q_percentile = np.percentile(np.absolute(np_weight), q)
    
    # Generate a binary mask same shape as weight to decide which element to prune
    bin_mask = np.where(np.absolute(np_weight) >= q_percentile, 1, 0)
        
    # Convert mask to torch tensor and put on GPU
    bin_mask = torch.from_numpy(bin_mask).to(device)
    
    # Multiply the weight by mask to perform pruning
    layer.weight.data = layer.weight.data * bin_mask
    
    return layer.weight.data

In [12]:
net.load_state_dict(torch.load("pretrained_model.pt"))

for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # change q value
        prune_by_percentage(layer, q=80.0)
        
        # Optional: Check the sparsity you achieve in each layer
        # Convert the weight of "layer" to numpy array
        np_weight = np.array(layer.weight.data.cpu())
        # Count number of zeros
        zeros = np.count_nonzero(np_weight==0)
        # Count number of parameters
        total = np_weight.size
        # Print sparsity
        print('Sparsity of '+name+': '+str(zeros/total))
        
test(net)

Sparsity of head_conv.0.conv: 0.7986111111111112
Sparsity of body_op.0.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.0.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.1.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.1.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.2.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.2.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.3.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.3.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.4.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.4.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.5.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.5.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.6.conv1.0.conv: 0.7999674479166666
Sparsity of body_op.6.conv2.0.conv: 0.7999945746527778
Sparsity of body_op.7.conv1.0.conv: 0.7999945746527778
Sparsity of body_op.7.conv2.0.conv: 0.7999945746527778
Sparsity of body_op.8.conv1.0.conv: 0.7999945746527778
Sparsity of body

### Lab2 (c) Finetune pruned model

In [4]:
def finetune_after_prune(net, trainloader, criterion, optimizer, prune=True):
    """
    Finetune the pruned model for a single epoch
    Make sure pruned weights are kept as zero
    """
    # Build a dictionary for the nonzero weights
    weight_mask = {}
    for name,layer in net.named_modules():
        if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
            # Your code here: generate a mask in GPU torch tensor to have 1 for nonzero element and 0 for zero element 
            bin_mask = np.where(np.array(layer.weight.data.cpu()) == 0, 0, 1)
            weight_mask[name] = torch.from_numpy(bin_mask).to(device)
    
    global_steps = 0
    train_loss = 0
    correct = 0
    total = 0
    start = time.time()
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        if prune:
            for name,layer in net.named_modules():
                if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
                    # Your code here: Use weight_mask to make sure zero elements remains zero
                    sparser_weights = layer.weight.data * weight_mask[name]
                    layer.weight.data = sparser_weights.float().to(device)
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        global_steps += 1

        if global_steps % 50 == 0:
            end = time.time()
            batch_size = 256
            num_examples_per_second = 50 * batch_size / (end - start)
            print("[Step=%d]\tLoss=%.4f\tacc=%.4f\t%.1f examples/second"
                 % (global_steps, train_loss / (batch_idx + 1), (correct / total), num_examples_per_second))
            start = time.time()
    
    

In [5]:
# Get pruned model
net.load_state_dict(torch.load("pretrained_model.pt"))
for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        prune_by_percentage(layer, q=80.0)

# Training setup, do not change
batch_size=256
lr=0.002
reg=1e-4

print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=16)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.875, weight_decay=reg, nesterov=False)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


In [19]:
# Model finetuning
for epoch in range(20):
    print('\nEpoch: %d' % epoch)
    net.train()
    finetune_after_prune(net, trainloader, criterion, optimizer)
    #Start the testing code.
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))

    if val_acc > best_acc:
        best_acc = val_acc
        print("Saving...")
        torch.save(net.state_dict(), "net_after_finetune.pt")


Epoch: 0
[Step=50]	Loss=0.9619	acc=0.6903	4743.8 examples/second
[Step=100]	Loss=0.8004	acc=0.7400	5488.7 examples/second
[Step=150]	Loss=0.7214	acc=0.7627	5399.9 examples/second
Test Loss=0.5548, Test acc=0.8191
Saving...

Epoch: 1
[Step=50]	Loss=0.4708	acc=0.8410	4230.1 examples/second
[Step=100]	Loss=0.4610	acc=0.8438	5047.1 examples/second
[Step=150]	Loss=0.4493	acc=0.8474	5545.3 examples/second
Test Loss=0.4861, Test acc=0.8400
Saving...

Epoch: 2
[Step=50]	Loss=0.4015	acc=0.8626	4257.4 examples/second
[Step=100]	Loss=0.3977	acc=0.8644	5358.7 examples/second
[Step=150]	Loss=0.3932	acc=0.8657	4672.9 examples/second
Test Loss=0.4563, Test acc=0.8506
Saving...

Epoch: 3
[Step=50]	Loss=0.3672	acc=0.8736	4590.3 examples/second
[Step=100]	Loss=0.3630	acc=0.8760	5377.9 examples/second
[Step=150]	Loss=0.3602	acc=0.8772	5171.1 examples/second
Test Loss=0.4349, Test acc=0.8571
Saving...

Epoch: 4
[Step=50]	Loss=0.3642	acc=0.8766	4532.7 examples/second
[Step=100]	Loss=0.3518	acc=0.8814	5147

In [20]:
# Check sparsity of the finetuned model, make sure it's not changed
net.load_state_dict(torch.load("net_after_finetune.pt"))

for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # Your code here:
        # Convert the weight of "layer" to numpy array
        np_weight = np.array(layer.weight.data.cpu())
        # Count number of zeros
        zeros = np.count_nonzero(np_weight==0)
        # Count number of parameters
        total = np_weight.size
        # Print sparsity
        print('Sparsity of '+name+': '+str(zeros/total))
        
test(net)

Sparsity of head_conv.0.conv: 0.7986111111111112
Sparsity of body_op.0.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.0.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.1.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.1.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.2.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.2.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.3.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.3.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.4.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.4.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.5.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.5.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.6.conv1.0.conv: 0.7999674479166666
Sparsity of body_op.6.conv2.0.conv: 0.7999945746527778
Sparsity of body_op.7.conv1.0.conv: 0.7999945746527778
Sparsity of body_op.7.conv2.0.conv: 0.7999945746527778
Sparsity of body_op.8.conv1.0.conv: 0.7999945746527778
Sparsity of body

### Lab2 (d) Iterative pruning

In [23]:
net.load_state_dict(torch.load("pretrained_model.pt"))
best_acc = 0.
for epoch in range(20):
    print('\nEpoch: %d' % epoch)
    
    net.train()
    if epoch<10:
        for name,layer in net.named_modules():
            if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
                # Increase model sparsity
                q = 8 * (epoch+1)
                prune_by_percentage(layer, q=q)
    if epoch<9:
        finetune_after_prune(net, trainloader, criterion, optimizer,prune=False)
    else:
        finetune_after_prune(net, trainloader, criterion, optimizer)
    
    #Start the testing code.
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))
    
    if epoch>=10:
        if val_acc > best_acc:
            best_acc = val_acc
            print("Saving...")
            torch.save(net.state_dict(), "net_after_iterative_prune.pt")


Epoch: 0
[Step=50]	Loss=0.0496	acc=0.9835	4290.6 examples/second
[Step=100]	Loss=0.0491	acc=0.9841	5099.3 examples/second
[Step=150]	Loss=0.0498	acc=0.9838	5466.7 examples/second
Test Loss=0.3238, Test acc=0.9145

Epoch: 1
[Step=50]	Loss=0.0522	acc=0.9826	4359.7 examples/second
[Step=100]	Loss=0.0496	acc=0.9838	5113.8 examples/second
[Step=150]	Loss=0.0496	acc=0.9839	5446.2 examples/second
Test Loss=0.3266, Test acc=0.9143

Epoch: 2
[Step=50]	Loss=0.0520	acc=0.9830	4380.5 examples/second
[Step=100]	Loss=0.0511	acc=0.9838	4935.9 examples/second
[Step=150]	Loss=0.0511	acc=0.9836	5094.2 examples/second
Test Loss=0.3297, Test acc=0.9127

Epoch: 3
[Step=50]	Loss=0.0592	acc=0.9809	4234.2 examples/second
[Step=100]	Loss=0.0583	acc=0.9810	5341.6 examples/second
[Step=150]	Loss=0.0572	acc=0.9810	5166.1 examples/second
Test Loss=0.3317, Test acc=0.9110

Epoch: 4
[Step=50]	Loss=0.0720	acc=0.9748	4497.0 examples/second
[Step=100]	Loss=0.0687	acc=0.9761	5421.8 examples/second
[Step=150]	Loss=0.068

In [24]:
# Check sparsity of the final model, make sure it's 80%
net.load_state_dict(torch.load("net_after_iterative_prune.pt"))

for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # change q value
        prune_by_percentage(layer, q=40.0)
        
        # Optional: Check the sparsity you achieve in each layer
        # Convert the weight of "layer" to numpy array
        np_weight = np.array(layer.weight.data.cpu())
        # Count number of zeros
        zeros = np.count_nonzero(np_weight==0)
        # Count number of parameters
        total = np_weight.size
        # Print sparsity
        print('Sparsity of '+name+': '+str(zeros/total))
        
test(net)

Sparsity of head_conv.0.conv: 0.7986111111111112
Sparsity of body_op.0.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.0.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.1.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.1.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.2.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.2.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.3.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.3.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.4.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.4.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.5.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.5.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.6.conv1.0.conv: 0.7999674479166666
Sparsity of body_op.6.conv2.0.conv: 0.7999945746527778
Sparsity of body_op.7.conv1.0.conv: 0.7999945746527778
Sparsity of body_op.7.conv2.0.conv: 0.7999945746527778
Sparsity of body_op.8.conv1.0.conv: 0.7999945746527778
Sparsity of body

### Lab2 (e) Global iterative pruning

In [6]:
def global_prune_by_percentage(net, q=80.0):
    """
    Pruning the weight paramters by threshold.
    :param q: pruning percentile. 'q' percent of the least 
    significant weight parameters will be pruned.
    """
    # A list to gather all the weights
    flattened_weights = []
    # Find global pruning threshold
    for name,layer in net.named_modules():
        if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
            # Convert weight to numpy
            np_weight = np.array(layer.weight.data.cpu())
            
            # Flatten the weight and append to flattened_weights
            flattened_weights.append(np_weight.flatten())
    
    # Concate all weights into a np array
    flattened_weights = np.concatenate(flattened_weights)
    # Find global pruning threshold
    thres = np.percentile(np.absolute(flattened_weights), q)
    
    # Apply pruning threshold to all layers
    for name,layer in net.named_modules():
        if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
            # Convert weight to numpy
            np_weight = np.array(layer.weight.data.cpu())
            
            # Generate a binary mask same shape as weight to decide which element to prune
            bin_mask = np.where(np.absolute(np_weight) >= thres, 1, 0)
            
            # Convert mask to torch tensor and put on GPU
            bin_mask = torch.from_numpy(bin_mask).to(device)
            
            # Multiply the weight by mask to perform pruning
            layer.weight.data = layer.weight.data * bin_mask
            

In [10]:
net.load_state_dict(torch.load("pretrained_model.pt"))
best_acc = 0.
for epoch in range(20):
    print('\nEpoch: %d' % epoch)
    q=(epoch+1)*8
    
    net.train()
    # Increase model sparsity
    if epoch<10:
        global_prune_by_percentage(net, q=q)
    if epoch<9:
        finetune_after_prune(net, trainloader, criterion, optimizer,prune=False)
    else:
        finetune_after_prune(net, trainloader, criterion, optimizer)
    
    #Start the testing code.
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))
    
    if epoch>=10:
        if val_acc > best_acc:
            best_acc = val_acc
            print("Saving...")
            torch.save(net.state_dict(), "net_after_global_iterative_prune.pt")


Epoch: 0
[Step=50]	Loss=0.0467	acc=0.9850	2485.7 examples/second
[Step=100]	Loss=0.0478	acc=0.9846	4671.5 examples/second
[Step=150]	Loss=0.0469	acc=0.9851	4546.3 examples/second
Test Loss=0.3259, Test acc=0.9138

Epoch: 1
[Step=50]	Loss=0.0482	acc=0.9845	3239.1 examples/second
[Step=100]	Loss=0.0500	acc=0.9842	4113.3 examples/second
[Step=150]	Loss=0.0484	acc=0.9847	4569.2 examples/second
Test Loss=0.3250, Test acc=0.9136

Epoch: 2
[Step=50]	Loss=0.0481	acc=0.9853	3279.7 examples/second
[Step=100]	Loss=0.0502	acc=0.9838	4598.0 examples/second
[Step=150]	Loss=0.0509	acc=0.9835	4969.4 examples/second
Test Loss=0.3245, Test acc=0.9136

Epoch: 3
[Step=50]	Loss=0.0496	acc=0.9851	3339.6 examples/second
[Step=100]	Loss=0.0500	acc=0.9849	4607.8 examples/second
[Step=150]	Loss=0.0499	acc=0.9846	4868.5 examples/second
Test Loss=0.3274, Test acc=0.9132

Epoch: 4
[Step=50]	Loss=0.0601	acc=0.9793	3584.7 examples/second
[Step=100]	Loss=0.0609	acc=0.9798	5270.6 examples/second
[Step=150]	Loss=0.059

In [11]:
net.load_state_dict(torch.load("net_after_global_iterative_prune.pt"))

zeros_sum = 0
total_sum = 0
for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # Your code here:
        # Convert the weight of "layer" to numpy array
        np_weight = np.array(layer.weight.data.cpu())
        # Count number of zeros
        zeros = np.count_nonzero(np_weight==0)
        # Count number of parameters
        total = np_weight.size
        zeros_sum+=zeros
        total_sum+=total
        print('Sparsity of '+name+': '+str(zeros/total))
print('Total sparsity of: '+str(zeros_sum/total_sum))
test(net)

Sparsity of head_conv.0.conv: 0.3101851851851852
Sparsity of body_op.0.conv1.0.conv: 0.6566840277777778
Sparsity of body_op.0.conv2.0.conv: 0.6384548611111112
Sparsity of body_op.1.conv1.0.conv: 0.6258680555555556
Sparsity of body_op.1.conv2.0.conv: 0.6480034722222222
Sparsity of body_op.2.conv1.0.conv: 0.6310763888888888
Sparsity of body_op.2.conv2.0.conv: 0.6692708333333334
Sparsity of body_op.3.conv1.0.conv: 0.6243489583333334
Sparsity of body_op.3.conv2.0.conv: 0.6886935763888888
Sparsity of body_op.4.conv1.0.conv: 0.7253689236111112
Sparsity of body_op.4.conv2.0.conv: 0.7822265625
Sparsity of body_op.5.conv1.0.conv: 0.7248263888888888
Sparsity of body_op.5.conv2.0.conv: 0.8130425347222222
Sparsity of body_op.6.conv1.0.conv: 0.7327473958333334
Sparsity of body_op.6.conv2.0.conv: 0.7643500434027778
Sparsity of body_op.7.conv1.0.conv: 0.7770182291666666
Sparsity of body_op.7.conv2.0.conv: 0.8261176215277778
Sparsity of body_op.8.conv1.0.conv: 0.8528103298611112
Sparsity of body_op.8.

### Lab 3 (b) and (c): Fixed-point quantization

In [ ]:
# Define quantized model and load weight
Nbits = 4 #Change this value to finish (b) and (c)

net = ResNetCIFAR(num_layers=20, Nbits=Nbits)
net = net.to(device)
net.load_state_dict(torch.load("pretrained_model.pt"))
test(net)


In [16]:
# Define quantized model and load weight
Nbits = 4 #Change this value to finish (b) and (c)

net = ResNetCIFAR(num_layers=20, Nbits=Nbits)
net = net.to(device)
net.load_state_dict(torch.load("pretrained_model.pt"))
test(net)


Files already downloaded and verified
Test Loss=0.3861, Test accuracy=0.8972


In [17]:
# Quantized model finetuning
finetune(net, epochs=20, batch_size=256, lr=0.002, reg=1e-4)   

# Load the model with best accuracy
net.load_state_dict(torch.load("quantized_net_after_finetune.pt"))
test(net)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified

Epoch: 0
[Step=50]	Loss=0.0695	acc=0.9753	3643.5 examples/second
[Step=100]	Loss=0.0694	acc=0.9755	4428.9 examples/second
[Step=150]	Loss=0.0678	acc=0.9757	4185.1 examples/second
Test Loss=0.3357, Test acc=0.9089
Saving...

Epoch: 1
[Step=200]	Loss=0.0756	acc=0.9727	1880.1 examples/second
[Step=250]	Loss=0.0616	acc=0.9789	4760.3 examples/second
[Step=300]	Loss=0.0595	acc=0.9799	4256.6 examples/second
[Step=350]	Loss=0.0594	acc=0.9799	4392.5 examples/second
Test Loss=0.3325, Test acc=0.9101
Saving...

Epoch: 2
[Step=400]	Loss=0.0602	acc=0.9814	1772.9 examples/second
[Step=450]	Loss=0.0650	acc=0.9775	4570.6 examples/second
[Step=500]	Loss=0.0606	acc=0.9797	4425.2 examples/second
[Step=550]	Loss=0.0598	acc=0.9795	4039.8 examples/second
Test Loss=0.3345, Test acc=0.9110
Saving...

Epoch: 3
[Step=600]	Loss=0.0598	acc=0.9805	1769.4 examples/second
[Step=650]	Loss=0.0581	acc=0.9814	4467.1 example

### Lab3 (d) Quantize pruned model

In [22]:
# Define quantized model and load weight
Nbits = 2 #Change this value to finish (d)

net = ResNetCIFAR(num_layers=20, Nbits=Nbits)
net = net.to(device)
net.load_state_dict(torch.load("net_after_global_iterative_prune.pt"))
test(net)


Files already downloaded and verified
Test Loss=7.6773, Test accuracy=0.1090


In [23]:
# Quantized model finetuning
finetune(net, epochs=20, batch_size=256, lr=0.002, reg=1e-4)

# Load the model with best accuracy
net.load_state_dict(torch.load("quantized_net_after_finetune.pt"))
test(net)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified

Epoch: 0
[Step=50]	Loss=2.3051	acc=0.1413	3317.1 examples/second
[Step=100]	Loss=2.2457	acc=0.1537	4217.5 examples/second
[Step=150]	Loss=2.2204	acc=0.1608	4396.4 examples/second
Test Loss=2.1529, Test acc=0.1698
Saving...

Epoch: 1
[Step=200]	Loss=2.1676	acc=0.1758	1671.0 examples/second
[Step=250]	Loss=2.1451	acc=0.1884	4527.0 examples/second
[Step=300]	Loss=2.1382	acc=0.1906	4057.7 examples/second
[Step=350]	Loss=2.1284	acc=0.2004	4184.5 examples/second
Test Loss=2.1173, Test acc=0.2061
Saving...

Epoch: 2
[Step=400]	Loss=2.1176	acc=0.2168	1691.8 examples/second
[Step=450]	Loss=2.1158	acc=0.2144	3946.6 examples/second
[Step=500]	Loss=2.1077	acc=0.2191	3963.3 examples/second
[Step=550]	Loss=2.1015	acc=0.2211	4413.4 examples/second
Test Loss=2.0996, Test acc=0.2257
Saving...

Epoch: 3
[Step=600]	Loss=2.0880	acc=0.2282	1657.9 examples/second
[Step=650]	Loss=2.0864	acc=0.2274	4221.5 example

### Lab3 (e) Prune quantized model

In [24]:
net = ResNetCIFAR(num_layers=20, Nbits=4)
net = net.to(device)

net.load_state_dict(torch.load("pretrained_model.pt"))
best_acc = 0.
for epoch in range(20):
    print('\nEpoch: %d' % epoch)
    q=(epoch+1)*8
    
    net.train()
    # Increase model sparsity
    if epoch<10:
        global_prune_by_percentage(net, q=q)
    if epoch<9:
        finetune_after_prune(net, trainloader, criterion, optimizer,prune=False)
    else:
        finetune_after_prune(net, trainloader, criterion, optimizer)
    
    #Start the testing code.
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))
    
    if epoch>=10:
        if val_acc > best_acc:
            best_acc = val_acc
            print("Saving...")
            torch.save(net.state_dict(), "quantized_net_after_global_iterative_prune.pt")


Epoch: 0
[Step=50]	Loss=0.0696	acc=0.9746	4202.5 examples/second
[Step=100]	Loss=0.0699	acc=0.9755	4973.2 examples/second
[Step=150]	Loss=0.0696	acc=0.9755	4925.8 examples/second
Test Loss=0.3433, Test acc=0.9070

Epoch: 1
[Step=50]	Loss=0.0775	acc=0.9730	3760.1 examples/second
[Step=100]	Loss=0.0772	acc=0.9735	4660.5 examples/second
[Step=150]	Loss=0.0743	acc=0.9743	4555.3 examples/second
Test Loss=0.3509, Test acc=0.9046

Epoch: 2
[Step=50]	Loss=0.0834	acc=0.9703	3924.2 examples/second
[Step=100]	Loss=0.0847	acc=0.9701	4429.4 examples/second
[Step=150]	Loss=0.0827	acc=0.9708	5240.3 examples/second
Test Loss=0.3646, Test acc=0.9031

Epoch: 3
[Step=50]	Loss=0.1089	acc=0.9617	3868.4 examples/second
[Step=100]	Loss=0.1063	acc=0.9624	4775.1 examples/second
[Step=150]	Loss=0.1076	acc=0.9617	4985.0 examples/second
Test Loss=0.3781, Test acc=0.8961

Epoch: 4
[Step=50]	Loss=0.1309	acc=0.9545	3908.8 examples/second
[Step=100]	Loss=0.1362	acc=0.9518	4735.9 examples/second
[Step=150]	Loss=0.134

In [25]:
net.load_state_dict(torch.load("quantized_net_after_global_iterative_prune.pt"))

zeros_sum = 0
total_sum = 0
for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        np_weight = layer.weight.data.cpu().numpy()
        zeros = (np_weight==0).sum()
        total = np_weight.flatten().shape[0]
        zeros_sum+=zeros
        total_sum+=total
        print('Sparsity of '+name+': '+str(zeros/total))
print('Total sparsity of: '+str(zeros_sum/total_sum))
test(net)

Sparsity of head_conv.0.conv: 0.30787037037037035
Sparsity of body_op.0.conv1.0.conv: 0.6575520833333334
Sparsity of body_op.0.conv2.0.conv: 0.6397569444444444
Sparsity of body_op.1.conv1.0.conv: 0.6280381944444444
Sparsity of body_op.1.conv2.0.conv: 0.6488715277777778
Sparsity of body_op.2.conv1.0.conv: 0.6315104166666666
Sparsity of body_op.2.conv2.0.conv: 0.6684027777777778
Sparsity of body_op.3.conv1.0.conv: 0.6252170138888888
Sparsity of body_op.3.conv2.0.conv: 0.6882595486111112
Sparsity of body_op.4.conv1.0.conv: 0.7261284722222222
Sparsity of body_op.4.conv2.0.conv: 0.7820095486111112
Sparsity of body_op.5.conv1.0.conv: 0.7249348958333334
Sparsity of body_op.5.conv2.0.conv: 0.8132595486111112
Sparsity of body_op.6.conv1.0.conv: 0.7331814236111112
Sparsity of body_op.6.conv2.0.conv: 0.7640245225694444
Sparsity of body_op.7.conv1.0.conv: 0.7770453559027778
Sparsity of body_op.7.conv2.0.conv: 0.8261176215277778
Sparsity of body_op.8.conv1.0.conv: 0.8527018229166666
Sparsity of bod